# WIP: Video generation

In [1]:
# We need it to import sunflower

import sys
sys.path.insert(0, "..")

# General

import numpy
import pandas

# Audio

from sunflower.sunflower.song_loader import Song, load_from_disk
from sunflower.sunflower.song_analyzer import SongAnalyzer
from sunflower.sunflower.utils import export_wav
from sunflower.sunflower.song_visualizer import visualize_waveform
import plotly.graph_objects as go
import numpy as np
import librosa
import soundfile as sf

# Video

from moviepy.editor import *
from moviepy.audio.AudioClip import AudioArrayClip
import moviepy
import pygame

# Autoreloading for easier development

%load_ext autoreload
%autoreload 2

## Audio

In [2]:
# Loading example file

raw_audio, extension = load_from_disk("assets/chillhop_mix/collab_dlj_72bpm.mp3")

song = Song(raw_audio, extension)

song.print_attributes()

waveform: [[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 ...
 [ 0.01153564 -0.01220703]
 [ 0.01165771 -0.01211548]
 [ 0.0093689  -0.01235962]], mono_waveform: [  0.   0.   0. ... -11.  -8. -49.], extension: mp3, channels: 2, sr: 44100, sample_width: 2


In [3]:
# Analyzing song

song_analyzer = SongAnalyzer(song)
song_analyzer.detect_tempo()

print(song_analyzer.tempo)

143.5546875


## Video

In [7]:
def squared_size(clip):

    # Resize
    clip = clip.crop(x_center=int(clip.w / 2),
                     y_center=int(clip.h / 2),
                     width=min(clip.w, clip.h),
                     height=min(clip.w, clip.h))

    return (clip)

In [8]:
audioclip = AudioArrayClip(song.waveform, fps=song.sr)

In [9]:
# Settings

WIDTH = 1080
HEIGHT = 1080
CENTER_PROP = 1

screensize = (WIDTH, HEIGHT)
prop_center = 0.8

BPM = 72
BPM_VID = 88

duration = 60 / BPM

artist_name = "DLJ x Nymano"
track_name = "WIP"

# Texts

textwidth = 0.2 * WIDTH
textheight = 0.2 * HEIGHT

In [47]:
60/88

0.6818181818181818

In [41]:
txtClip_artist = TextClip(artist_name,
                          color='white',
                          font="Helvetica",
                          kerning=5,
                          size=(textwidth, None))

txtClip_track = TextClip(track_name,
                         color='LightPink',
                         font="Helvetica-bold",
                         kerning=5,
                         size=(textwidth, None))

# Load all the videos

# Chillhop vid

clip = VideoFileClip("assets/chillhop_mix/mpc_vid_89bpm.mp4", fps_source="fps")
clip_fps = clip.fps

clip = clip.speedx(factor=BPM / BPM_VID)
clip = squared_size(clip)
clip = clip.resize(screensize).resize(CENTER_PROP)

# Looping the clip
clip = clip.loop(n=4)

# Say that you want it to appear 10s at the center of the screen
txtClip_artist = txtClip_artist.set_pos(
    (WIDTH / 2 - txtClip_artist.w / 2,
     0.1 * HEIGHT)).set_start(duration * 2).set_duration(duration * 2)

# Say that you want it to appear 10s at the center of the screen
txtClip_track = txtClip_track.set_pos(
    (WIDTH / 2 - txtClip_track.w / 2,
     0.25 * HEIGHT)).set_start(duration * 2).set_duration(duration * 2)

# Overlay the text clip on the first video clip
video = CompositeVideoClip(
    [clip.set_position("center"), txtClip_artist, txtClip_track])

video = video.set_audio(audioclip)

# Cut de la video

video = video.subclip(0, 15)

In [42]:
video.preview(fps=10, audio=True)

In [43]:
pygame.quit()

In [ ]:
#video.ipython_display(fps=20, autoplay=1,maxduration=30)

## Saving video

In [44]:
video.write_videofile("results/test_chilhop.mp4", temp_audiofile="temp-audio.m4a", remove_temp=True, codec="libx264", audio_codec="aac")

t:   0%|          | 0/375 [00:00<?, ?it/s, now=None]                

Moviepy - Building video results/test_chilhop.mp4.
MoviePy - Writing audio in temp-audio.m4a
MoviePy - Done.
Moviepy - Writing video results/test_chilhop.mp4



Moviepy - Done !
Moviepy - video ready results/test_chilhop.mp4
